In [7]:
import numpy as np
cycle_days=100000./24./3600.
print(cycle_days)
shortdepth=1500.
longdepth=1600.
holddepth = 1500.
speed=0.12
surftime_min=30.
extra_time=2*3600.
TCR = cycle_days*24*3600.
# time hold long
THL = TCR-(longdepth+(longdepth-holddepth))/speed-surftime_min*60+extra_time
TCHLB = shortdepth/speed
TCHLE = THL-holddepth/speed-1000.
TDPHL = THL +(longdepth-holddepth)/speed + 3600.
TDPHS = shortdepth/speed+1800.

print('TimeCycleHoldLongBeg = %d' % TCHLB)
print('TimeCycleHoldLongEnd = %d'% TCHLE);
print('TimeCycleRep = %d'% TCR);
print('TimeDescentProfHoldLong = %d' % TDPHL);
print('TimeDescentProfHoldShort = %d'%TDPHS);
print('TimeHoldLong = %d'%THL)

1.15740740741
TimeCycleHoldLongBeg = 12500
TimeCycleHoldLongEnd = 77733
TimeCycleRep = 100000
TimeDescentProfHoldLong = 95666
TimeDescentProfHoldShort = 14300
TimeHoldLong = 91233


In [124]:
cycle = 10*24*3600 #s this is what we'd like (we won't get it)
print cycle
surftime = 30.*60.
holddepth = 1583. # get this from a simulation.  It is set by `PistonFollowDefault`
PrBotHoldLong = 1610.
speed = 0.12

# how deep can we go to get the inertial pair?
dt = 14.35*3600./2.
print dt
dt = dt-surftime # subtract the transmit
PrBotHoldShort = dt*speed/2.
print('PrBotHoldLong: %1.0f dbar'% np.round(PrBotHoldLong))
print('PrBotHoldShort: %1.0f dbar'% np.round(PrBotHoldShort))

# get TimeHoldLong
TimeHoldLong = cycle-np.abs(PrBotHoldLong-holddepth)/speed \
  -PrBotHoldLong/speed \
            -surftime
print(PrBotHoldLong/speed)
print('TimeHoldLong: %1.0f'%np.round(TimeHoldLong))

# get the maximum TimeCycleHoldLongBeg
TCHLB = 2.*PrBotHoldShort/speed
print('TimeCycleHoldLongBeg should be less than %d'%round(TCHLB))
print('TimeDescentProfHoldLong should be greater than %d'%round(cycle))



864000
25830.0
PrBotHoldLong: 1610 dbar
PrBotHoldShort: 1442 dbar
13416.6666667
TimeHoldLong: 848558
TimeCycleHoldLongBeg should be less than 24030
TimeDescentProfHoldLong should be greater than 864000


## Compare two files:

In [59]:
def getParams(ifile):
    mp=dict()
    ifp = open(ifile, mode='rt')
    
    for line in ifp:
        if line.find('=') > 0:
            if '#' in line:
                i = line.find('#')
                line = line[0:i]
            line = line.strip()
            if line.find('=') > 0:
                tok = line.split('=')
                if len(tok)!=2:
                    sys.exit()

                varnam=tok[0].strip()
                varval=tok[1].strip()
                mp[varnam]=varval
    return mp


In [83]:
mpjmk=getParams('EMAPEXLabSea15/MissionParams/jmklabseaparams.txt')
mprf=getParams('EMAPEXLabSea15/MissionParams/Argo_labsea3_mission.txt')
#print mpjmk

print 'THESE ARE DIFFERENT  JMK vs RF'
for key,value in sorted(mpjmk.items()):
    if key in mprf.keys():
        if value!=mprf[key]:
            print ('%30s'%key),'=',value,'  vs  ',mprf[key]
    else:
#        print key
        pass
print 'NOT IN JMK'
for key,value in sorted(mprf.items()):
    if not(key in mpjmk.keys()):
        print key,value
print 'NOT IN RF'
for key,value in sorted(mpjmk.items()):
    if not(key in mprf.keys()):
        print key,value
print 'THESE ARE THE SAME!'
for key,value in sorted(mpjmk.items()):
    if key in mprf.keys():
        if value==mprf[key]:
            print ('%30s'%key),'=',value,'  vs  ',mprf[key]
    else:
#        print key
        pass

THESE ARE DIFFERENT  JMK vs RF
              EmaProcessNslide = 40   vs   25
               EmaProcessNvals = 40   vs   50
                    FloatAlpha = -3.673e-06   vs   -0.000003673
                     FloatBeta = 3.2e-05   vs   0.000032
                    OptodeMode = 0   vs   1
            PistonParkPosition = 42   vs   63
         PistonStoragePosition = 42   vs   16
          PressureMaxRecordRaw = 500   vs   100
                  PressurePark = 1000   vs   1500
                  RawEfSaveRep = 30   vs   10
          SalinityBallastPoint = 34.7   vs   34.6
       TemperatureBallastPoint = 0.4   vs   1.8
          TimeCycleHoldLongEnd = 167733   vs   160000
NOT IN JMK
2DurationFastProfiling 43200
NOT IN RF
ConsoleBaudRate 4800
CtdPmin 5
CtdSampleDelPr1 2
CtdSampleDelPr2 10
CtdSampleDelPr3 20
CtdSampleDelPr4 50
CtdSampleN1 1100
CtdSampleN2 20
CtdSampleN3 20
CtdSampleN4 20
CtdSamplePrFirst 2
CtdType 4
DateCycleStarted 2014/11/03 19:49:42
DateNextLevel2 2014/11/04 08:49:42
DateP

In [47]:
mpd=_asdict(mp)

NameError: name '_asdict' is not defined

## Prettify files

In [120]:
def prettify(ifile,ofile):
    mp=dict()
    ifp = open(ifile, mode='rt')
    ofp = open(ofile, mode='wt')
    
    for line in ifp:
        if line.find('=') > 0:
            tok = line.split('=',1)
            line.split()
            if len(tok)!=2:
                sys.exit()

            varnam=tok[0].strip()
            varval=tok[1].strip()
            st = varnam
            stlen = len(st)
            for i in range(27-stlen):
                st = st+' '
            mp[varnam]=st+'= '+tok[1][:-1]
    
    for key,value in sorted(mp.items()):
        ofp.write(value+'\n')
    ofp.close()
    ifp.close()

In [121]:
mp=prettify('EMAPEXLabSea15/MissionParams/Argo_labsea3_mission.txt','EMAPEXLabSea15/MissionParams/LabseaMission001.txt',)